In [1]:
import pandas as pd
import numpy as np
import psycopg2 

In [75]:
conn = psycopg2.connect(
    host="fit3164db.cxkhqsoitzhb.ap-southeast-2.rds.amazonaws.com", 
    port=5432,
    user="postgres",
    password="fit3164d13edas",
    database = "testdb"
)
conn.autocommit = True
cursor = conn.cursor()

In [76]:
cursor.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'actuals'")
acc_cols = cursor.fetchall()
acc_cols = [item[0] for item in acc_cols]

cursor.execute("select * from actuals")
actual_data = cursor.fetchall()

In [77]:
cursor.close()
conn.close()

In [102]:
#data = pd.DataFrame(actual_data, columns= acc_cols)
#data.sort_values(by=['id'], inplace=True)
data = pd.read_csv('Compiled.csv', parse_dates=['datetime'])

In [200]:
MAE = {
    'naive_forecast':           [0 for i in range(0,51)],
    'seasonal_forecast_daily':  [0 for i in range(0,51)],
    'seasonal_forecast_weekly': [0 for i in range(0,51)],
    'seasonal_forecast_yearly': [0 for i in range(0,51)],
    'random_walk_forecast':     [0 for i in range(0,51)],
    'linear_regression':        [0 for i in range(0,51)]}

MAPE = {
    'naive_forecast':           [0 for i in range(0,51)],
    'seasonal_forecast_daily':  [0 for i in range(0,51)],
    'seasonal_forecast_weekly': [0 for i in range(0,51)],
    'seasonal_forecast_yearly': [0 for i in range(0,51)],
    'random_walk_forecast':     [0 for i in range(0,51)],
    'linear_regression':        [0 for i in range(0,51)]}

RMSE = {
    'naive_forecast':           [0 for i in range(0,51)],
    'seasonal_forecast_daily':  [0 for i in range(0,51)],
    'seasonal_forecast_weekly': [0 for i in range(0,51)],
    'seasonal_forecast_yearly': [0 for i in range(0,51)],
    'random_walk_forecast':     [0 for i in range(0,51)],
    'linear_regression':        [0 for i in range(0,51)]}

In [184]:
forecasts = ['naive_forecast', 'seasonal_forecast_daily', 'seasonal_forecast_weekly', 
             'seasonal_forecast_yearly', 'random_walk_forecast', 'linear_regression']

In [174]:
exogenous_columns = ['pressure_f', 'cloud_cov_f', 'temp_f', 'wind_dir_f', 'wind_sp_f', 
                     'tue','wed','thu','fri','sat','sun',
                     'feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec',
                     'h1','h2','h3','h4','h5','h6','h7','h8','h9','h10','h11','h12',
                     'h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']

In [163]:
end_of_train = int(len(data)*0.9)
# Where the train/validate split occurs

In [172]:
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.preprocessing import StandardScaler

In [201]:
num_forecasts = int((len(data) - end_of_train)//24 - 1)

for f in tqdm(range(0, num_forecasts)):
    
    frontier        = end_of_train + 24*f
    train           = data[:frontier]
    train_linear    = data[frontier-24*183:frontier] # Train on six months
    test            = data[frontier:frontier + 48]
    
    y_hat = test[['load']].copy()
    y_hat['naive_forecast']           = train['load'].iloc[-1]
    y_hat['seasonal_forecast_daily']  = [train['load'].iloc[-(i%24 + 1)] for i in range(47,-1,-1)]
    y_hat['seasonal_forecast_weekly'] = train['load'][-24*7:-24*7+48].values
    y_hat['seasonal_forecast_yearly'] = train['load'][-24*365:-24*365+48].values
    drift                             = (train['load'].iloc[-1]-train['load'].iloc[0])/(365*24-1)
    y_hat['random_walk_forecast']     = y_hat['naive_forecast']+[i*drift for i in range(1,49)]
    
    if not f%14: #Retrain every 14 days
        
        forecaster = ForecasterAutoreg(
                     regressor        = LinearRegression(),
                     lags             = [1, 2, 24, 25, 48, 49, 72, 73, 96, 97, 120, 121, 144, 145, 167, 168],
                     transformer_y    = StandardScaler(),
                     transformer_exog = None
                 )

        forecaster.fit(y=train_linear['load'], exog=train_linear[exogenous_columns])
    
    y_hat['linear_regression'] = forecaster.predict(48, train['load'], test[exogenous_columns])
    
    
    for forecast in forecasts:
        abs_err              = (abs(y_hat[forecast]-y_hat['load']))
        MAE [forecast][0:48] = np.add(MAE[forecast][0:48],abs_err/num_forecasts)
        RMSE[forecast][0:48] = np.add(RMSE[forecast][0:48],abs_err**2/num_forecasts)
        MAPE[forecast][0:48] = np.add(MAPE[forecast][0:48],abs_err/(y_hat['load']*num_forecasts))

100%|████████████████████████████████████████████████████████████████████████████████| 139/139 [00:06<00:00, 22.81it/s]


In [202]:
for forecast in forecasts:
    
    MAE[forecast][48]    = np.mean(MAE[forecast][0:24])
    MAE[forecast][49]    = np.mean(MAE[forecast][24:48])
    MAE[forecast][50]    = np.mean(MAE[forecast][0:48])
    
    MAPE[forecast][48]   = np.mean(MAPE[forecast][0:24])
    MAPE[forecast][49]   = np.mean(MAPE[forecast][24:48])
    MAPE[forecast][50]   = np.mean(MAPE[forecast][0:48])
    
    RMSE[forecast][0:48] = np.sqrt(RMSE[forecast][0:48])
    RMSE[forecast][48]   = np.mean(RMSE[forecast][0:24])
    RMSE[forecast][49]   = np.mean(RMSE[forecast][24:48])
    RMSE[forecast][50]   = np.mean(RMSE[forecast][0:48])

MAE_df  = pd.DataFrame(MAE,  index=['H'+str(i) for i in range(1,49)] + ['First 24 hours'] + ['Second 24 hours'] + ['Total'])
MAPE_df = pd.DataFrame(MAPE, index=['H'+str(i) for i in range(1,49)] + ['First 24 hours'] + ['Second 24 hours'] + ['Total'])
RMSE_df = pd.DataFrame(RMSE, index=['H'+str(i) for i in range(1,49)] + ['First 24 hours'] + ['Second 24 hours'] + ['Total'])

In [198]:
pd.options.display.float_format = '{:20,.3f}'.format

In [203]:
frames = [MAE_df .iloc[50:51], 
          MAPE_df.iloc[50:51], 
          RMSE_df.iloc[50:51]]
results = pd.concat(frames, keys=["MAE", "MAPE", "RMSE"])
results

,,naive_forecast,seasonal_forecast_daily,seasonal_forecast_weekly,seasonal_forecast_yearly,random_walk_forecast,linear_regression
MAE,Total,"91,888.363","42,076.110","26,793.517","121,357.706","91,904.359","26,485.077"
MAPE,Total,0.080,0.038,0.024,0.110,0.080,0.024
RMSE,Total,"101,004.828","56,824.442","36,241.256","142,803.227","101,027.406","35,489.072"


In [168]:
filename = r"Benchmark_Results.csv"
results.to_csv(filename, encoding='utf-8', index=False, header=True)